In [ ]:
import numpy as np
import pandas as pd
import string
import pickle
import spacy
nlp_id = spacy.blank('id')
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [ ]:
df = pd.read_csv("dataset/News.csv")
df.isnull().sum()
# Dropping the rows having None values
df = df.dropna(subset=['content'])
# Resetting the indices
df = df.reset_index(drop=True)

## Preprocessing

In [ ]:
def sentence_stem(token_DocID):
    new_token = []
    for to_ in token_DocID:
        new_token.append( ( stemmer.stem(to_) ))
    return new_token

In [ ]:
def nlp_tokenization(contents):
    list_tokens_from_docs = []
    for content in contents:
        nlp_contents = nlp_id(content.lower())
        clean_token = []
        for token_of_nlp_contents in nlp_contents:
            # remove punctuation & remove stopword
            if not token_of_nlp_contents.is_digit and not token_of_nlp_contents.is_punct \
                and not token_of_nlp_contents.is_stop:
                # clean_token.append(str(token_of_nlp_contents))
                clean_token.append(token_of_nlp_contents)
                # print(token_of_nlp_contents)
        list_tokens_from_docs.append(clean_token)
    return list_tokens_from_docs

In [ ]:
nlp_stem = sentence_stem(df['content'][0:30])
nlp_tokens = nlp_tokenization(nlp_stem)
# nlp_tokens = nlp_tokenization(df['content'])

## Pairing Term and Document ID

In [ ]:
def pair_token_docID(nlp_tokens):
    doc_list_ofToken_DocID = []
    # index_docID
    for docID, doc_list_T in enumerate(nlp_tokens):
        # doc_list_ofToken_DocID_post = []
        for doc_list in doc_list_T:
            # doc_list_ofToken_DocID_post.append((doc_list, docID+1))
            # doc_list_ofToken_DocID.append((doc_list, docID+1))
            doc_list_ofToken_DocID.append((doc_list.text, docID+1))
    return doc_list_ofToken_DocID

def remove_space(nlp_tokens_tup):
    new_tokens = []
    for i in nlp_tokens_tup:
        if not i[0].isspace():
            new_tokens.append(i)
    return new_tokens

def sort_token_docID(tup):
    return(sorted(tup, key = lambda x: x[0]))

In [ ]:
nlp_tokens_docId = pair_token_docID(nlp_tokens)
nlp_tokens_docId_clean_space = remove_space(nlp_tokens_docId)
nlp_tokens_docId_sorted = sort_token_docID(nlp_tokens_docId_clean_space)
nlp_tokens_docId_sorted

## Inverted Index

In [ ]:
from collections import deque

In [ ]:
def inverted_index(tokens_docID):
    vocab = {}
    for token, docId in tokens_docID:
        if not token in vocab:
            vocab[token] = deque()
            vocab[token].append(docId)
        else:
            temp_post_list = vocab[token]
            temp_post_list.append(docId)
            # prevent duplication
            temp_post_list = set(temp_post_list)
            # back to linkedlist
            temp_post_list = deque(temp_post_list)
            vocab[token] = temp_post_list
    vocab_freq = {}
    # pairing term and doc frequency
    for key, val in vocab.items():
        vocab_freq[(key, len(val))] = val
    return vocab_freq

In [ ]:
dict_inverted_index = inverted_index(nlp_tokens_docId_sorted)

In [ ]:
dict_inverted_index